In [1]:
from transformers import BertModel, DistilBertModel
bert_model = BertModel.from_pretrained('monologg/kobert')
distilbert_model = DistilBertModel.from_pretrained('monologg/distilkobert')

c:\Users\user\anaconda3\envs\nof\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
## 키워드 작업한 데이터 불러오기
import pandas as pd
import torch

df_fraud = pd.read_csv('data/df_fraud_keyword.csv', index_col=False)

In [6]:
df_fraud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4182 entries, 0 to 4181
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   판례일련번호         4182 non-null   int64 
 1   사건명            4180 non-null   object
 2   사건번호           4182 non-null   object
 3   선고일자           4182 non-null   object
 4   법원명            4182 non-null   object
 5   사건종류명          4182 non-null   object
 6   판례상세링크         4182 non-null   object
 7   판례정보일련번호       4182 non-null   int64 
 8   선고             4182 non-null   object
 9   판결유형           4182 non-null   object
 10  판시사항           3028 non-null   object
 11  판결요지           2482 non-null   object
 12  참조조문           3001 non-null   object
 13  참조판례           1808 non-null   object
 14  판례내용           4182 non-null   object
 15  판례내용_상단        4182 non-null   object
 16  판례내용_이유        4178 non-null   object
 17  판례내용_이유_전처리    4182 non-null   object
 18  판례내용_이유_불용어제거  4178 non-null

# 승민님 BERT모델(판시사항)

In [11]:
df_fraud['모델용_문장'] = "[CLS]" + df_fraud['판시사항'] + "[SEP]"

In [13]:
df_fraud['모델용_문장'].iloc[1]

'[CLS] 배심원이 참여하는 형사재판, 즉 국민참여재판을 거쳐 제1심법원이 배심원의 만장일치 무죄평결을 받아들여 피고인에 대하여 무죄판결을 선고한 경우, ‘증거의 취사 및 사실의 인정’에 관한 제1심법원의 판단은 한층 더 존중될 필요가 있는지 여부(적극) 및 이때 제1심법원의 무죄판결에 대한 항소심에서의 추가적이거나 새로운 증거조사의 범위[SEP]'

In [16]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from kobert_transformers import get_tokenizer

bert_model = BertModel.from_pretrained('monologg/kobert')
distilbert_model = DistilBertModel.from_pretrained('monologg/distilkobert')

# Define your model
model = distilbert_model  # 모델 선언
tokenizer = get_tokenizer()  # 토크나이저 선언

for i in range(10):
    sentences = []
    sentences.append(df_fraud['모델용_문장'].iloc[i])


# 각 문장을 토큰화, id화
tokenized_sentences = [tokenizer.tokenize(sentence) for sentence in sentences]
tokenid_list = [tokenizer.convert_tokens_to_ids(tokens) for tokens in tokenized_sentences]

# sequence를 같은 길이로 padding
max_length = max(len(ids) for ids in tokenid_list)
padded_tokenid_list = [ids + [0] * (max_length - len(ids)) for ids in tokenid_list]

# Longtensor로 dtype변경
tokenid_tensor = torch.LongTensor(padded_tokenid_list)

# Print the tensor
print(tokenid_tensor)

# 추론
tensor_output = model(tokenid_tensor)  # 모델에 위의 전처리 과정을 거친 자연어를 input으로 넣는다
print(tensor_output[0].shape)  # Output shape
print(tensor_output[0])  # Output 확인

# 출력은 (batchsize, 토큰id의 길이, 768). 여기서 768은 임베딩 공간의 크기

c:\Users\user\anaconda3\envs\nof\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--monologg--kobert. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tensor([[   2, 3893, 5478, 6335, 4128,  150, 7253, 4128,   93, 7846, 6903, 4092,
         7828, 4071, 7202, 3480, 5384, 7096, 4596, 5898, 3162, 2682, 3893,  517,
         7656, 5781, 7088, 4955, 2587, 1528, 6006,  833, 2322, 4128,  225, 7253,
         4128,  116, 7925, 6896, 1835, 4466, 5889, 7318, 3310,   18, 7202, 5543,
           40,    3]])
torch.Size([1, 50, 768])
tensor([[[ 0.0527,  0.2446,  0.1086,  ..., -0.5052, -0.5448,  0.1624],
         [ 0.6586,  0.0192,  0.1596,  ...,  0.0030, -0.1197,  0.2213],
         [ 0.4585, -0.0808,  0.4904,  ...,  0.1427, -0.2438, -0.2900],
         ...,
         [ 0.6087, -0.0853,  0.3526,  ..., -0.3582, -0.3065, -0.0323],
         [ 0.1492,  0.0111,  0.4568,  ..., -0.2438,  0.0845, -0.7004],
         [ 0.0525,  0.2449,  0.1089,  ..., -0.5057, -0.5442,  0.1627]]],
       grad_fn=<NativeLayerNormBackward0>)
